# **Importing**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# **Building CNN Model**

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64*5*5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x,2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x,2)
        x = x.view(x.size(0),-1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)



# **Load MNIST**

In [6]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

trainset = torchvision.datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

testset = torchvision.datasets.MNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)
trainloader = DataLoader(trainset,batch_size=64,shuffle=True)
testloader = DataLoader(testset,batch_size=64)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 483kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.37MB/s]


# **Training**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model =CNN().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001)
loss_fn=nn.CrossEntropyLoss()

for epoch in range(8):
  model.train()
  total_loss=0
  for images,labels in trainloader:
    images,labels =images.to(device),labels.to(device)
    optimizer.zero_grad()
    output=model(images)
    loss=loss_fn(output,labels)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()

  print(f"Epoch {epoch+1}, Loss: {total_loss/len(trainloader):.4f}")


Epoch 1, Loss: 0.1352


# **Testing Accuracy**

In [ ]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        out = model(images)
        pred = out.argmax(1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

print(f"Accuracy: {100*correct/total:.2f}%")


# **Saving the Model**

In [ ]:
torch.save(model.state_dict(), "mnist_cnn.pth")